In [1]:
setwd("/Users/pranavkulkarni/SDM/Climate_Models_Arenaviruses/")
library(raster)
library(terra)
library(tidyverse)

Loading required package: sp

terra 1.7.78

── Attaching core tidyverse packages ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::extract() masks terra::extract(), raster::extract()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ dplyr::select()  masks raster::select()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
curr_bclims <- rast(list.files(path = "./Data/Input/Processed/Resampled/guan",
                     full.names =  T))

ssp2_bclims <- rast(list.files(path = "./Data/Input/Processed/Projected/guan/SSP2",
                     full.names =  T))

ssp5_bclims <- rast(list.files(path = "./Data/Input/Processed/Projected/guan/SSP5",
                     full.names =  T))

In [3]:
curr_foi <- rast("./Data/Output/iterations/S_alstoni/current/foi.tif")
ssp2_foi <- rast("./Data/Output/iterations/S_alstoni/ssp2/foi_ssp2.tif")
ssp5_foi <- rast("./Data/Output/iterations/S_alstoni/ssp5/foi_ssp5.tif")

In [4]:
delta_ssp2_bclims <- ssp2_bclims - curr_bclims
delta_ssp5_bclims <- ssp5_bclims - curr_bclims


delta_ssp2_foi <- ssp2_foi - curr_foi
delta_ssp5_foi <- ssp5_foi - curr_foi

## SSP2

In [5]:
mat_delta_ssp2_foi <- as.matrix(values(delta_ssp2_foi))
mat_delta_ssp2_bioclims <- as.matrix(values(delta_ssp2_bclims))
delta_ssp2_mod_df <- as.data.frame(cbind(mat_delta_ssp2_foi, mat_delta_ssp2_bioclims))
delta_ssp2_mod_df <- delta_ssp2_mod_df %>% 
  select(-c(ndvi, dem)) %>% 
  mutate(foi = floor(gpw_v4_2000*1000)) %>% 
  select(-gpw_v4_2000) %>%
  drop_na(.)

rm(mat_delta_ssp2_bioclims,mat_delta_ssp2_foi)

In [6]:
if (!dir.exists(file.path("./Data/Output/Iterations/S_alstoni/ssp2/foi_mod_plots"))) {dir.create("./Data/Output/Iterations/S_alstoni/ssp2/foi_mod_plots")}

In [7]:
model_delta_ssp2 <- lm(foi ~., data = delta_ssp2_mod_df)

summary(model_delta_ssp2)



Call:
lm(formula = foi ~ ., data = delta_ssp2_mod_df)

Residuals:
    Min      1Q  Median      3Q     Max 
-36.462  -2.633   0.282   2.753  40.841 

Coefficients:
                           Estimate Std. Error t value Pr(>|t|)    
(Intercept)               3.151e+00  2.096e-01  15.034  < 2e-16 ***
bclim_01                 -6.612e-01  2.252e-01  -2.936 0.003328 ** 
bclim_02                 -3.273e-01  3.033e-01  -1.079 0.280615    
bclim_03                 -6.046e-01  3.542e-02 -17.069  < 2e-16 ***
bclim_04                  1.371e-01  6.579e-03  20.839  < 2e-16 ***
bclim_05                  4.029e+04  1.201e+04   3.354 0.000798 ***
bclim_06                 -4.029e+04  1.201e+04  -3.354 0.000798 ***
bclim_07                 -4.029e+04  1.201e+04  -3.354 0.000797 ***
bclim_08                 -6.282e-01  6.173e-02 -10.176  < 2e-16 ***
bclim_09                  2.307e-01  5.577e-02   4.136 3.54e-05 ***
bclim_10                 -1.880e+00  2.685e-01  -7.002 2.52e-12 ***
bclim_11            

In [8]:
for (i in 1:24) {
  
  pl <- ggplot(delta_ssp2_mod_df, aes(x = delta_ssp2_mod_df[,i], y = delta_ssp2_mod_df[,25])) + 
    geom_point(alpha = 0.3) +
    labs(x = names(delta_ssp2_mod_df)[i], y = "FOI x 1000", subtitle = "red: lm, green: binomial, blue: gam") +
    stat_smooth(method = "lm", col = "red", se = T) +
    stat_smooth(method = "auto", col = "blue", se = T) +
    theme_classic()
  ggsave(filename = paste0("./Data/Output/Iterations/S_alstoni/ssp2/foi_mod_plots/foi_mod_", i, ".png"), 
         plot = pl, device = "png",
         width = 11, height = 8.5, units = "in")
  
}

`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ 

## SSP5

In [9]:
mat_delta_ssp5_foi <- as.matrix(values(delta_ssp5_foi))
mat_delta_ssp5_bioclims <- as.matrix(values(delta_ssp5_bclims))
delta_ssp5_mod_df <- as.data.frame(cbind(mat_delta_ssp5_foi, mat_delta_ssp5_bioclims))
delta_ssp5_mod_df <- delta_ssp5_mod_df %>% 
  select(-c(ndvi, dem)) %>% 
  mutate(foi = floor(gpw_v4_2000*1000)) %>% 
  select(-gpw_v4_2000) %>%
  drop_na(.)

rm(mat_delta_ssp5_bioclims,mat_delta_ssp5_foi)

In [10]:
if (!dir.exists(file.path("./Data/Output/Iterations/S_alstoni/ssp5/foi_mod_plots"))) {dir.create("./Data/Output/Iterations/S_alstoni/ssp5/foi_mod_plots")}

In [11]:
model_delta_ssp5 <- lm(foi ~., data = delta_ssp5_mod_df)

summary(model_delta_ssp5)



Call:
lm(formula = foi ~ ., data = delta_ssp5_mod_df)

Residuals:
    Min      1Q  Median      3Q     Max 
-35.613  -2.846   0.129   2.778  44.363 

Coefficients:
                           Estimate Std. Error t value Pr(>|t|)    
(Intercept)               4.559e+00  2.076e-01  21.959  < 2e-16 ***
bclim_01                 -2.870e+00  1.809e-01 -15.868  < 2e-16 ***
bclim_02                  4.153e+00  2.368e-01  17.539  < 2e-16 ***
bclim_03                 -8.838e-01  2.621e-02 -33.722  < 2e-16 ***
bclim_04                  9.539e-02  5.771e-03  16.529  < 2e-16 ***
bclim_05                  8.541e+03  1.228e+04   0.695 0.486832    
bclim_06                 -8.541e+03  1.228e+04  -0.695 0.486836    
bclim_07                 -8.548e+03  1.228e+04  -0.696 0.486510    
bclim_08                 -7.560e-01  5.757e-02 -13.133  < 2e-16 ***
bclim_09                  9.450e-01  4.968e-02  19.022  < 2e-16 ***
bclim_10                 -2.566e-01  2.429e-01  -1.056 0.290780    
bclim_11            

In [ ]:

for (i in 1:24) {
  
  pl <- ggplot(delta_ssp5_mod_df, aes(x = delta_ssp5_mod_df[,i], y = delta_ssp5_mod_df[,25])) + 
    geom_point(alpha = 0.3) +
    labs(x = names(delta_ssp5_mod_df)[i], y = "FOI x 1000", subtitle = "red: lm, green: binomial, blue: gam") +
    stat_smooth(method = "lm", col = "red", se = T) +
    stat_smooth(method = "auto", col = "blue", se = T) +
    theme_classic()
  ggsave(filename = paste0("./Data/Output/Iterations/S_alstoni/ssp5/foi_mod_plots/foi_mod_", i, ".png"), 
         plot = pl, device = "png",
         width = 11, height = 8.5, units = "in")
  
}

`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using method = 'gam' and formula = 'y ~ s(x, bs = "cs")'
